# Python Transcrição Fonética

Passo a passo da elaboração do ficheiro python e da aplicação das expressões regulares da transcrição fonética ao corpus 3.

Se necessário, instalar pacotes `pandas`, `regex` e `odfpy`.

In [ ]:
#!pip install pandas
#!pip install regex
#!pip install odfpy

Importar as bibliotecas necessárias para o processamento: `pandas` para a manipulação de dados, `regex` para as expressões regulares com lookbehind variáveis, e `IPython.display` para a visualização dos resultados no notebook.

In [2]:
import pandas as pd
import regex as re  # Use the regex module for variable-width look-behinds
from IPython.display import display

Definir a função `load_column_to_list`, que lê a coluna A do ficheiro ODS e devolve a lista de palavras.

In [3]:
def load_column_to_list(file_path, sheet_name):
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
        column_data = df[0].dropna().tolist()
        return column_data
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
    except ValueError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Definir a função `load_regex_to_dict`, que lê as expressões regulares e os respetivos IDs e substituições a partir de uma folha ODS.

In [4]:
def load_regex_to_dict(file_path, sheet_name):
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
        regex_dict = {str(k): {"replacement": str(v), "id": str(i)} for k, v, i in zip(df[0], df[1], df[2])}
        return regex_dict
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
    except ValueError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Definir a função `apply_patterns_to_words` que aplica todas as expressões regulares às palavras da lista. Gera um ficheiro de log com o histórico de transformações e regista os IDs das regras aplicadas.

In [5]:
def apply_patterns_to_words(words_list, regex_dict, log_file_path="../Debug/debug_fonetica_notebook.txt"):
    results = []
    with open(log_file_path, "w", encoding="utf-8") as log_file:
        for word in words_list:
            processed_word = word
            used_ids = []
            for pattern, data in regex_dict.items():
                replacement = data["replacement"]
                rule_id = data["id"]
                try:
                    if replacement == "nan":
                        replacement = ""
                    new_word = re.sub(pattern, replacement, processed_word)
                    if new_word != processed_word:
                        used_ids.append(rule_id)
                        log_file.write(f"Origin Word: {processed_word} -> Processed Word: {new_word}\n")
                        log_file.write(f"Pattern Applied: {pattern}->{replacement} (Pattern id: {rule_id})\n\n")
                    processed_word = new_word
                except re.error as e:
                    print(f"Regex error for pattern '{pattern}': {e}")
            results.append((processed_word, used_ids))
    return results

Definir os caminhos para o ficheiro ODS e os nomes das folhas que contêm a lista de palavras e as expressões regulares.

In [6]:
file_path = "../ODS/Fonetica.ods"
folha_palavras = "Palavras"
folha_expressoes = "ExpressoesFonetica"

Carregar a lista de palavras e o dicionário de expressões regulares a partir do ficheiro ODS.

In [7]:
words_list = load_column_to_list(file_path, folha_palavras)
regex_dict = load_regex_to_dict(file_path, folha_expressoes)

Mostrar a lista de palavras e o dicionário de expressões carregados.

In [8]:
print(words_list)
print(regex_dict)

['ˈalto', 'amaˈrelo', 'aˈmigo', 'anˈtigo', 'aˈssado', 'aˈzul', 'ˈbaixo', 'ˈbelo', 'ˈbom', 'boˈnito', 'ˈbranco', 'caˈpaz', 'ˈcaro', 'casˈtanho', 'ˈcheio', 'chiˈnês', 'ˈclaro', 'comˈprido', 'conˈtente', 'difeˈrente', 'diˈfícil', 'ˈdoce', 'eˈnorme', 'esˈcuro', 'esˈtranho', 'ˈfácil', 'faˈmoso', 'feˈliz', 'ˈfino', 'ˈforte', 'ˈfresco', 'ˈfrio', 'ˈgrande', 'iˈgual', 'imporˈtante', 'ˈjunto', 'ˈleve', 'ˈlimpo', 'ˈlindo', 'ˈlivre', 'ˈlongo', 'ˈmau', 'natuˈral', 'ˈnegro', 'norˈmal', 'ˈnovo', 'obriˈgado', 'ˈótimo', 'paˈssado', 'peˈqueno', 'ˈpobre', 'portuˈguês', 'poˈssível', 'preˈciso', 'ˈpreto', 'princiˈpal', 'ˈpronto', 'ˈquente', 'ˈrico', 'ˈseco', 'simˈpático', 'ˈsimples', 'soˈzinho', 'ˈtriste', 'ˈúltimo', 'ˈvelho', 'verˈmelho', 'ˈvivo', 'aˈgora', 'aˈí', 'aˈinda', 'aˈli', 'amaˈnhã', 'aˈpenas', 'aˈqui', 'aˈssim', 'aˈté', 'basˈtante', 'ˈbem', 'ˈcá', 'ˈcedo', 'demasiˈado', 'deˈpois', 'deˈpressa', 'emˈbora', 'enˈtão', 'entreˈtanto', 'finalˈmente', 'geralˈmente', 'ˈhoje', 'iˈmenso', 'ˈjá', 'ˈlá', 'ˈl

Aplicar as expressões regulares às palavras, registando os resultados processados e os IDs das regras usadas.

In [9]:
if words_list and regex_dict:
    processed_data = apply_patterns_to_words(words_list, regex_dict)
    results_for_df = []
    for original, (processed, ids_used) in zip(words_list, processed_data):
        results_for_df.append({
            "Original": original,
            "Processado": processed,
            "IDs usados": ", ".join(ids_used) if ids_used else ""
        })
    df_resultados = pd.DataFrame(results_for_df)

## Resultados

Apresentar os resultados da transcrição fonética: palavra original, palavra processada e IDs das expressões regulares aplicadas.

In [10]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 0):
    display(df_resultados)

,Original,Processado,IDs usados
0,ˈalto,[ˈaɫtu],"T8, T43, T92, T93"
1,amaˈrelo,[ɐmɐˈɾ/E/lu],"T6, T8, T46, T55, T90, T92, T93"
2,aˈmigo,[ɐˈmiɣu],"T8, T46, T69, T92, T93"
3,anˈtigo,[ɐ̃ˈtiɣu],"T8, T19, T69, T92, T93"
4,aˈssado,[ɐˈsaðu],"T8, T46, T64, T76, T92, T93"
5,aˈzul,[ɐˈzuɫ],"T44, T46, T92, T93"
6,ˈbaixo,[ˈbajʃu],"T8, T31, T61, T92, T93"
7,ˈbelo,[ˈb/E/lu],"T8, T55, T92, T93"
8,ˈbom,[ˈbõ],"T23, T92, T93"
9,boˈnito,[buˈnitu],"T5, T8, T92, T93"


## Instalar bibliotecas necessárias (se necessário)

Remova os `#` abaixo se precisar instalar os pacotes.

## Imports

## Carregar Coluna para Lista

## Carregar Regras Regex para Dicionário

## Aplicar os Padrões às Palavras

## Definição dos Ficheiros

## Carregar Dados

## Processar Palavras com os Padrões

## Resultados